In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [2]:
req = requests.get('https://www.selver.ee/')

In [4]:
soup = BS4(req.content, features='html.parser')

In [5]:
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')

In [6]:
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':url_class})

In [7]:
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96&p=" for i in find_tag]

In [ ]:
get_urls

In [ ]:
#Kogu html linkide kogust
product_html = requests.get(get_urls[5])
#puhasta ilusa seebiga
soup = BS4(product_html.content, features='html.parser')
#leia toote ja hinna kategoorijad li tagist
find_product_group = soup.find_all('li',{'class':product_class})
#leia hinnad
find_product_group_prices = soup.find_all('span',{'class':'price'})
#võta linki pealkirjad ehk toote nimed
find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
#leia kõik hinnad
prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]

In [92]:
html_list = list()
list_of_products = list()
list_of_prices = list()
dict_of_products = dict()

In [93]:
#start of the urls and end of the URLS
for z in range(5,23):
    #Find product last page
    product_html = requests.get(get_urls[z])
    #puhasta ilusa seebiga
    soup = BS4(product_html.content, features='html.parser')
    try:
        find_product_last_page = soup.find('a',{'class':'last'}).text
    except AttributeError:
        find_product_last_page = 3
    
    for i in range(1,int(find_product_last_page)+1):
        product_html = requests.get(get_urls[z]+str(i))
        soup = BS4(product_html.content, features='html.parser')
        html_list.append(soup)  
        print(f"Page URL: {get_urls[z]+str(i)} Last page is: {find_product_last_page}")

    for y in range(len(html_list)):  
        find_product_group = html_list[y].find_all('li',{'class':product_class})
        find_product_group_prices = html_list[y].find_all('span',{'class':'price'})
        find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
        prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]
        for t,f in zip(find_product_group_titles,prices):
            dict_of_products.update({t:re.split(r'\s+([0-9]+.[0-9]+\s+./[A-z]+)',f)[:2]})
          


In [69]:
from sqlalchemy import create_engine, insert, update, MetaData, Table, String, Column, Integer

In [101]:
table_conn = create_engine('sqlite:///selver_data.db', echo=True)
toode = []
toode = [{'Toode':x,'Hind':y[0]} for x,y in dict_of_products.items()]

In [102]:
conn = table_conn.connect()
meta = MetaData(table_conn)
table = Table(
               'ViimaneKatseNr13', meta,
               Column('id', Integer, primary_key=True, autoincrement=True),
               Column('Toode', String),
               Column('Hind', String),
               Column('Hind/Kogus', String),
)
meta.create_all()

conn.execute(table.insert(),toode)


2021-03-03 22:07:45,184 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-03 22:07:45,185 INFO sqlalchemy.engine.base.Engine ()
2021-03-03 22:07:45,187 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-03 22:07:45,188 INFO sqlalchemy.engine.base.Engine ()
2021-03-03 22:07:45,192 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("ViimaneKatseNr13")
2021-03-03 22:07:45,194 INFO sqlalchemy.engine.base.Engine ()
2021-03-03 22:07:45,234 INFO sqlalchemy.engine.base.Engine INSERT INTO "ViimaneKatseNr13" ("Toode", "Hind") VALUES (?, ?)
2021-03-03 22:07:45,235 INFO sqlalchemy.engine.base.Engine (('Banaan Chiquita, kg', '1,39 €'), ('Kurk Luunja, kg', '5,99 €'), ('Kirssploomtomat, 250 g', '1,29 €'), ('Kirsskobartomat, 500 g', '2,99 €'), ('Apelsin Navel, kg', '1,59 €'), ('Frillis lõigatud, tk', '1,99 €'), ('Porgand, LAHEOTSA, 500 g', '1,29 €'), ('Õun Golden pakitud, 1 kg', '0,59 €')  .

In [ ]:
#Finish regex!!!!!!!!!!!!!!!!!
prices = re.split(r'\s+([0-9]+.[0-9]+\s[A-z]+)','2,29 €  38,17 €/l ')
prices

In [100]:
toode

[{'Toode': 'Banaan Chiquita, kg', 'Hind': ['1,39 €', '1,39 €/kg']},
 {'Toode': 'Kurk Luunja, kg', 'Hind': ['5,99 €', '5,99 €/kg']},
 {'Toode': 'Kirssploomtomat, 250 g', 'Hind': ['1,29 €', '5,16 €/kg']},
 {'Toode': 'Kirsskobartomat, 500 g', 'Hind': ['2,99 €', '5,98 €/kg']},
 {'Toode': 'Apelsin Navel, kg', 'Hind': ['1,59 €', '1,59 €/kg']},
 {'Toode': 'Frillis lõigatud, tk', 'Hind': ['1,99 €', '1,99 €/kg']},
 {'Toode': 'Porgand, LAHEOTSA, 500 g', 'Hind': ['1,29 €', '1,29 €/tk']},
 {'Toode': 'Õun Golden pakitud, 1 kg', 'Hind': ['0,59 €', '1,18 €/kg']},
 {'Toode': 'Kobartomat, kg', 'Hind': ['1,29 €', '1,29 €/kg']},
 {'Toode': 'Avokaado karbis Ready to Eat, 300 g',
  'Hind': ['1,79 €', '1,79 €/kg']},
 {'Toode': 'Pirn Conference, 1 kg', 'Hind': ['2,99 €', '9,97 €/kg']},
 {'Toode': 'Õun Jonagold, kg', 'Hind': ['2,49 €', '2,49 €/kg']},
 {'Toode': 'Mugulsibul pakitud võrgus, 1 kg', 'Hind': ['0,99 €', '0,99 €/kg']},
 {'Toode': 'Sidrun, kg', 'Hind': ['1,29 €', '1,29 €/kg']},
 {'Toode': 'Õun Kanzi,

10379

[{'Toode': 'Banaan Chiquita, kg', 'Hind': '1,39 €'},
 {'Toode': 'Kurk Luunja, kg', 'Hind': '5,99 €'},
 {'Toode': 'Kirssploomtomat, 250 g', 'Hind': '1,29 €'},
 {'Toode': 'Kirsskobartomat, 500 g', 'Hind': '2,99 €'},
 {'Toode': 'Apelsin Navel, kg', 'Hind': '1,59 €'},
 {'Toode': 'Frillis lõigatud, tk', 'Hind': '1,99 €'},
 {'Toode': 'Porgand, LAHEOTSA, 500 g', 'Hind': '1,29 €'},
 {'Toode': 'Õun Golden pakitud, 1 kg', 'Hind': '0,59 €'},
 {'Toode': 'Kobartomat, kg', 'Hind': '1,29 €'},
 {'Toode': 'Avokaado karbis Ready to Eat, 300 g', 'Hind': '1,79 €'},
 {'Toode': 'Pirn Conference, 1 kg', 'Hind': '2,99 €'},
 {'Toode': 'Õun Jonagold, kg', 'Hind': '2,49 €'},
 {'Toode': 'Mugulsibul pakitud võrgus, 1 kg', 'Hind': '0,99 €'},
 {'Toode': 'Sidrun, kg', 'Hind': '1,29 €'},
 {'Toode': 'Õun Kanzi, kg', 'Hind': '0,89 €'},
 {'Toode': 'Lehtsalat potis, GRÜNE FEE, tk', 'Hind': '1,59 €'},
 {'Toode': 'Melon kollane, kg', 'Hind': '1,99 €'},
 {'Toode': 'Õun Royal Gala, kg', 'Hind': '2,29 €'},
 {'Toode': 'Lillkaps